# Importing Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import os
import glob
import matplotlib.pyplot as plt
import math
import cv2


%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import statsmodels.api as sm

# Importing Data

In [3]:
complete_data = pd.read_excel("right_nail_data_with_AgeGender.xlsx")

In [4]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   number   287 non-null    int64  
 1   mean_r   287 non-null    float64
 2   mean_g   287 non-null    float64
 3   mean_b   287 non-null    float64
 4   mean_rg  287 non-null    float64
 5   HHR      287 non-null    float64
 6   Ent      287 non-null    float64
 7   B        287 non-null    float64
 8   G1       287 non-null    float64
 9   G2       287 non-null    float64
 10  G3       287 non-null    float64
 11  G4       287 non-null    float64
 12  G5       287 non-null    float64
 13  age      287 non-null    int64  
 14  gender   287 non-null    int64  
 15  label    287 non-null    float64
dtypes: float64(13), int64(3)
memory usage: 36.0 KB


In [5]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(complete_data, test_size=0.3)

In [6]:
# Separate features and target variable in train data
X_train = train_data.drop(columns=['number','label'])
y_train = train_data['label']

# Separate features and target variable in test data
X_test = test_data.drop(columns=['number','label'])
y_test = test_data['label']

# Algorithms

In [7]:
algorithms = {
    'Linear Regression': LinearRegression(),
    'SVM Regression': SVR(kernel='poly'),  # Adjust kernel as needed
    'RandomForest': RandomForestRegressor(),
    'Gradient Boost': GradientBoostingRegressor(),
    'knn': KNeighborsRegressor(),
    'LGBM': LGBMRegressor(),
    'CatBoost': CatBoostRegressor(),
    'Kernel Ridge Regressor': KernelRidge(),
    'Elastic Net': ElasticNet(),
    'Bayesian Ridge': BayesianRidge(),
    'XG Boost': XGBRegressor()
}

In [8]:
# Metric tables
metric_table_train = pd.DataFrame()
metric_table_test = pd.DataFrame()

# Training and Testing

In [9]:
# Run the algorithms ... create metrics and plots
for algorithm_name, algorithm in algorithms.items():

    # Train model
    algorithm.fit(X_train, y_train)

    # Train predictions
    y_train_pred = algorithm.predict(X_train)

    # Test predictions
    y_test_pred = algorithm.predict(X_test)

    # Train metrics
    r2_train = r2_score(y_train, y_train_pred)
    mse_train = mean_squared_error(y_train, y_train_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)

    # Test metrics
    r2_test = r2_score(y_test, y_test_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)

    # Additional metrics using statsmodels for all algorithms
    residuals_train = y_train - y_train_pred
    residuals_test = y_test - y_test_pred

    durbin_watson_stat_train = sm.stats.durbin_watson(residuals_train)
    jb_stat_train, jb_p_value_train, _, _ = sm.stats.jarque_bera(residuals_train)

    durbin_watson_stat_test = sm.stats.durbin_watson(residuals_test)
    jb_stat_test, jb_p_value_test, _, _ = sm.stats.jarque_bera(residuals_test)

    # Update metric tables
    metric_table_train.at[algorithm_name, 'MAE'] = mae_train
    metric_table_train.at[algorithm_name, 'R-squared'] = r2_train
    metric_table_train.at[algorithm_name, 'MSE'] = mse_train
    metric_table_train.at[algorithm_name, 'Durbin-Watson'] = durbin_watson_stat_train
    metric_table_train.at[algorithm_name, 'Jarque-Bera'] = jb_stat_train
    metric_table_train.at[algorithm_name, 'JB P-value'] = jb_p_value_train

    metric_table_test.at[algorithm_name, 'MAE'] = mae_test
    metric_table_test.at[algorithm_name, 'R-squared'] = r2_test
    metric_table_test.at[algorithm_name, 'MSE'] = mse_test
    metric_table_test.at[algorithm_name, 'Durbin-Watson'] = durbin_watson_stat_test
    metric_table_test.at[algorithm_name, 'Jarque-Bera'] = jb_stat_test
    metric_table_test.at[algorithm_name, 'JB P-value'] = jb_p_value_test


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000303 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 840
[LightGBM] [Info] Number of data points in the train set: 200, number of used features: 14
[LightGBM] [Info] Start training from score 11.614000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

# Results

In [10]:
# Display metrics in tables
print("Metrics - Train Data:\n")
print(metric_table_train.to_string())
print("-------------------------------------------------")

print("Metrics - Test Data:\n")
print(metric_table_test.to_string())

Metrics - Train Data:

                             MAE  R-squared           MSE  Durbin-Watson  Jarque-Bera    JB P-value
Linear Regression       1.223451   0.342592  2.557190e+00       2.145689   121.710191  3.723640e-27
SVM Regression          1.432182   0.128690  3.389224e+00       2.223158     4.005388  1.349712e-01
RandomForest            0.492840   0.890315  4.266517e-01       2.185356    25.967709  2.297120e-06
Gradient Boost          0.411180   0.930389  2.707726e-01       2.355889     2.010055  3.660346e-01
knn                     1.389700   0.238986  2.960194e+00       2.162458     0.152457  9.266043e-01
LGBM                    0.547465   0.865005  5.251046e-01       2.268664    81.246865  2.277547e-18
CatBoost                0.099523   0.996029  1.544720e-02       2.449950     1.879557  3.907145e-01
Kernel Ridge Regressor  1.414806   0.152339  3.297235e+00       2.118303    12.265625  2.170468e-03
Elastic Net             1.498421   0.116419  3.436958e+00       2.194834     

# To Classification

--> <10.5 --> Anemic

--> >=10.5 ---> Non-Anemic 

In [32]:
algorithm = CatBoostRegressor()

In [33]:
# Train model
algorithm.fit(X_train, y_train)

# Test predictions
y_test_pred = algorithm.predict(X_test)

Learning rate set to 0.03175
0:	learn: 1.9529553	total: 5.23ms	remaining: 5.22s
1:	learn: 1.9328992	total: 8.89ms	remaining: 4.44s
2:	learn: 1.9178255	total: 12.7ms	remaining: 4.23s
3:	learn: 1.9032887	total: 13.7ms	remaining: 3.41s
4:	learn: 1.8863798	total: 17.3ms	remaining: 3.44s
5:	learn: 1.8724355	total: 19.8ms	remaining: 3.29s
6:	learn: 1.8552512	total: 23.4ms	remaining: 3.32s
7:	learn: 1.8387990	total: 26.5ms	remaining: 3.28s
8:	learn: 1.8271829	total: 27.6ms	remaining: 3.03s
9:	learn: 1.8162020	total: 28.5ms	remaining: 2.82s
10:	learn: 1.8054561	total: 30.6ms	remaining: 2.75s
11:	learn: 1.7950166	total: 31.7ms	remaining: 2.61s
12:	learn: 1.7817157	total: 35.9ms	remaining: 2.72s
13:	learn: 1.7728390	total: 36.8ms	remaining: 2.59s
14:	learn: 1.7591138	total: 40.7ms	remaining: 2.67s
15:	learn: 1.7453057	total: 44.3ms	remaining: 2.73s
16:	learn: 1.7320107	total: 48.2ms	remaining: 2.79s
17:	learn: 1.7228464	total: 50.9ms	remaining: 2.78s
18:	learn: 1.7122102	total: 55.1ms	remaining:

In [34]:
actual_labels = (y_test<10.5).tolist()
pred_labels = (pd.Series(y_test_pred)<10.5).tolist()

In [35]:
TP=TN=FN=FP = 0
for i in range(len(actual_labels)):
    if(actual_labels[i]==True and pred_labels[i]==True):
        TP +=1
    if(actual_labels[i]==False and pred_labels[i]==False):
        TN +=1
    if(actual_labels[i]==True and pred_labels[i]==False):
        FN +=1
    if(actual_labels[i]==False and pred_labels[i]==True):
        FP +=1

print(f"Accuracy = {(TP+TN)/(TP+TN+FP+FN)}")
print(f"Precision = {(TP)/(TP+FP)}")
print(f"Sensitivity = {(TP)/(TP+FN)}")
print(f"Specificity = {(TN)/(TN+FP)}")

Accuracy = 0.7011494252873564
Precision = 0.4375
Sensitivity = 0.2916666666666667
Specificity = 0.8571428571428571


# LEAVE ONE OUT

In [11]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt

In [12]:
cv = LeaveOneOut()

In [13]:
X = complete_data.drop(columns=['number','label'])
y = complete_data['label']

## XGB Regression

In [14]:
model = XGBRegressor()

#use LOOCV to evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

In [15]:
y_true = y
y_pred = y + scores

In [16]:
print(f"MAE {mean(absolute(scores))}")
print(f"RMSE {sqrt(mean(absolute(scores)))}")
print(f"R-squared {r2_score(y_true,y_pred)}")

MAE 1.3698635991857442
RMSE 1.1704117220814836
R-squared 0.23126879137698608


## CatBoost Regression

In [29]:
model = CatBoostRegressor()

#use LOOCV to evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

In [30]:
y_true = y
y_pred = y + scores

In [31]:
print(f"MAE {mean(absolute(scores))}")
print(f"RMSE {sqrt(mean(absolute(scores)))}")
print(f"R-squared {r2_score(y_true,y_pred)}")

MAE 1.2767278286589028
RMSE 1.1299238154224835
R-squared 0.3021297123141824


## RandomForest

In [26]:
model = RandomForestRegressor()

#use LOOCV to evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

In [27]:
y_true = y
y_pred = y + scores

In [28]:
print(f"MAE {mean(absolute(scores))}")
print(f"RMSE {sqrt(mean(absolute(scores)))}")
print(f"R-squared {r2_score(y_true,y_pred)}")

MAE 1.287484320557491
RMSE 1.1346736625821061
R-squared 0.29787061726974995


## LGBM

In [17]:
model = LGBMRegressor()

#use LOOCV to evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

In [18]:
y_true = y
y_pred = y + scores

In [19]:
print(f"MAE {mean(absolute(scores))}")
print(f"RMSE {sqrt(mean(absolute(scores)))}")
print(f"R-squared {r2_score(y_true,y_pred)}")

MAE 1.30513419504137
RMSE 1.1424246999436636
R-squared 0.264286894343034


## Linear Regression

In [20]:
model = LinearRegression()

#use LOOCV to evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

In [21]:
y_true = y
y_pred = y + scores

In [22]:
print(f"MAE {mean(absolute(scores))}")
print(f"RMSE {sqrt(mean(absolute(scores)))}")
print(f"R-squared {r2_score(y_true,y_pred)}")

MAE 1.328753018537109
RMSE 1.1527154976563423
R-squared 0.24122648070511554


# PyCaret

In [23]:
import pycaret

In [24]:
from pycaret.regression import *
s = setup(complete_data, target='label', ignore_features=['number'], preprocess=False, session_id=123)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Regression
3,Original data shape,"(287, 16)"
4,Transformed data shape,"(287, 15)"
5,Transformed train set shape,"(200, 15)"
6,Transformed test set shape,"(87, 15)"
7,Ignore features,1
8,Numeric features,14


In [25]:
best_r = compare_models(sort = 'R2', n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,1.2980,2.7486,1.6222,0.2619,0.1376,0.1221,0.1190
et,Extra Trees Regressor,1.3158,2.9238,1.6723,0.2146,0.1410,0.1233,0.0900
ada,AdaBoost Regressor,1.3532,2.9737,1.6872,0.2038,0.1435,0.1285,0.0470
catboost,CatBoost Regressor,1.3303,2.9535,1.6895,0.2029,0.1425,0.1248,1.9380
ridge,Ridge Regression,1.3289,3.0277,1.6963,0.1832,0.1435,0.1264,0.0160
lr,Linear Regression,1.3306,3.0463,1.7004,0.1778,0.1438,0.1266,0.3560
br,Bayesian Ridge,1.3654,3.0905,1.7231,0.1684,0.1452,0.1291,0.0160
xgboost,Extreme Gradient Boosting,1.3905,3.1311,1.7423,0.1336,0.1465,0.1291,0.0640
gbr,Gradient Boosting Regressor,1.3840,3.1662,1.7454,0.1314,0.1478,0.1308,0.0590
lightgbm,Light Gradient Boosting Machine,1.3593,3.1696,1.7494,0.1239,0.1487,0.1286,0.0510
